## Import libraries

In [1]:
import pandas as pd
import urllib.request
import zipfile


### Download the results file from the TSE Dados Abertos

Uncomment when running the code for the first time, then you can comment again

In [2]:
# url_download = 'https://cdn.tse.jus.br/estatistica/sead/odsele/votacao_secao/votacao_secao_2022_BR.zip'
# opener = urllib.request.build_opener()
# opener.addheaders = [('User-agent', 'Mozilla/5.0')]
# urllib.request.install_opener(opener)
# urllib.request.urlretrieve(url_download, f"CSV/votacao_secao_2022_BR.zip")

If you prefer you can download the files manually: The full list of files can be found [here](https://dadosabertos.tse.jus.br/dataset/resultados-2022)!

### Unzip the downloaded file


Uncomment when running the code for the first time, then you can comment again

In [3]:
# with zipfile.ZipFile(f"CSV/votacao_secao_2022_BR.zip", 'r') as zip_ref:
#     zip_ref.extractall("CSV/")

### Read the extracted CSV

In [4]:
df = pd.read_csv("CSV/votacao_secao_2022_BR.csv", encoding="Latin 1", sep=";", usecols=['NR_TURNO','SG_UF','NM_MUNICIPIO','NR_ZONA','NR_SECAO','DS_CARGO','NM_VOTAVEL','QT_VOTOS'])

In [5]:
list_states = df['SG_UF'].unique()
# df['SG_UF'].unique()
df_co = df[df['SG_UF'].isin(['GO','MT','MS','DF'])]

In [6]:
# Quality check, total votes for Lula in Bahia
# df[(df['SG_UF']=='BA')&(df['NM_VOTAVEL']=='LUIZ INÁCIO LULA DA SILVA')&(df['DS_CARGO']=='PRESIDENTE')&(df['NR_TURNO']==2)]['QT_VOTOS'].sum()

### Filter by 2o turno and exclude null and branco votes, 

In [8]:
df_co = df_co[(df_co['NR_TURNO']==2)].copy()

## RUN PIVOT

In [9]:
results_co = df_co[['SG_UF','NM_MUNICIPIO','NM_VOTAVEL','QT_VOTOS']].groupby(by=['SG_UF','NM_MUNICIPIO','NM_VOTAVEL']).sum("QT_VOTOS")
pivot_co = results_co.unstack(level='NM_VOTAVEL')
pivot_co

QT_VOTOS  \
NM_VOTAVEL                             JAIR MESSIAS BOLSONARO   
SG_UF NM_MUNICIPIO                                              
DF    BRASÍLIA                                        1041331   
GO    ABADIA DE GOIÁS                                    5394   
      ABADIÂNIA                                          5759   
      ACREÚNA                                            6597   
      ADELÂNDIA                                          1517   
...                                                       ...   
MT    VERA                                               3847   
      VILA BELA DA SANTÍSSIMA TRINDADE                   4157   
      VILA RICA                                          6914   
      VÁRZEA GRANDE                                     84427   
      ÁGUA BOA                                          11324   

                                                                              \
NM_VOTAVEL                             LUIZ INÁCIO LULA DA SILVA VOTO BRANCO   
SG_UF NM_MUNICIPIO                                                             
DF    BRASÍLIA                                            729295       29663   
GO    ABADIA DE GOIÁS                                       3814          79   
      ABADIÂNIA                                             2774          81   
      ACREÚNA                                               5210         102   
      ADELÂNDIA                                              651          11   
...                                                          ...         ...   
MT    VERA                                                  1495          46   
      VILA BELA DA SANTÍSSIMA TRINDADE                      3140          47   
      VILA RICA                                             2984          60   
      VÁRZEA GRANDE                                        59268        1420   
      ÁGUA BOA                                              3117          93   

                                                  
NM_VOTAVEL                             VOTO NULO  
SG_UF NM_MUNICIPIO                                
DF    BRASÍLIA                             38203  
GO    ABADIA DE GOIÁS                        110  
      ABADIÂNIA                              155  
      ACREÚNA                                185  
      ADELÂNDIA                               27  
...                                          ...  
MT    VERA                                    71  
      VILA BELA DA SANTÍSSIMA TRINDADE       111  
      VILA RICA                              140  
      VÁRZEA GRANDE                         2343  
      ÁGUA BOA                               141  

[467 rows x 4 columns]

In [10]:
results_co_sessao = df_co[['SG_UF','NM_MUNICIPIO','NR_ZONA','NR_SECAO','NM_VOTAVEL','QT_VOTOS']].groupby(by=['SG_UF','NM_MUNICIPIO','NR_ZONA','NR_SECAO','NM_VOTAVEL']).sum("QT_VOTOS")
pivot_co_sessao = results_co_sessao.unstack(level='NM_VOTAVEL')
pivot_co_sessao

QT_VOTOS  \
NM_VOTAVEL                          JAIR MESSIAS BOLSONARO   
SG_UF NM_MUNICIPIO NR_ZONA NR_SECAO                          
DF    BRASÍLIA     1       1                         120.0   
                           2                         114.0   
                           3                         120.0   
                           4                         115.0   
                           5                         111.0   
...                                                    ...   
MT    ÁGUA BOA     30      98                        152.0   
                           99                        131.0   
                           100                       177.0   
                           101                        99.0   
                           102                       127.0   

                                                                           \
NM_VOTAVEL                          LUIZ INÁCIO LULA DA SILVA VOTO BRANCO   
SG_UF NM_MUNICIPIO NR_ZONA NR_SECAO                                         
DF    BRASÍLIA     1       1                             90.0         3.0   
                           2                             96.0         4.0   
                           3                             79.0        10.0   
                           4                             80.0         6.0   
                           5                             87.0         5.0   
...                                                       ...         ...   
MT    ÁGUA BOA     30      98                            46.0         NaN   
                           99                            58.0         1.0   
                           100                           59.0         2.0   
                           101                           52.0         3.0   
                           102                           39.0         1.0   

                                               
NM_VOTAVEL                          VOTO NULO  
SG_UF NM_MUNICIPIO NR_ZONA NR_SECAO            
DF    BRASÍLIA     1       1              8.0  
                           2              9.0  
                           3              4.0  
                           4              8.0  
                           5              5.0  
...                                       ...  
MT    ÁGUA BOA     30      98             NaN  
                           99             NaN  
                           100            2.0  
                           101            3.0  
                           102            4.0  

[35932 rows x 4 columns]

## Export PIVOT

In [11]:
pivot_co_sessao.to_excel("Output/pivot_co_sessao_2turno.xlsx", merge_cells=False)

In [ ]:
# df_results = df_ne[['SG_UF','NM_MUNICIPIO']].copy()
# df_results['Lula_abslt'] = ""
# df_results['Bolsonaro_abslt'] = ""
# df_results['Lula_pctg'] = ""
# df_results['Bolsonaro_pctg'] = ""
# df_results['VotosTotais'] = ""
# df_results.drop_duplicates(inplace=True)
# df_results = df_results.sort_values(by='SG_UF')
# df_results

In [ ]:
# for i,r in df_results.iterrows():
#     print(r['SG_UF'],r['NM_MUNICIPIO'])
#     uf = r['SG_UF']
#     muni = r['NM_MUNICIPIO']
#     results = df_ne[(df_ne['SG_UF']==uf)&(df_ne['NM_MUNICIPIO']==muni)][['NM_VOTAVEL','QT_VOTOS']].groupby(by='NM_VOTAVEL').sum("QT_VOTOS")
#     results['pctg'] = (results.QT_VOTOS / results.QT_VOTOS.sum())*100
#     df_results.at[i,'Lula_pctg'] = results[results.index=='LUIZ INÁCIO LULA DA SILVA']['pctg'][0]
#     df_results.at[i,'Lula_abslt'] = results[results.index=='LUIZ INÁCIO LULA DA SILVA']['QT_VOTOS'][0]
#     bolsonaro_pctg = results[results.index=='JAIR MESSIAS BOLSONARO']['pctg'][0]
#     bolsonaro_abslt = results[results.index=='JAIR MESSIAS BOLSONARO']['QT_VOTOS'][0]    
#     df_results.at[i,'Bolsonaro_pctg'] = bolsonaro_pctg
#     df_results.at[i,'Bolsonaro_abslt'] = bolsonaro_abslt
#     df_results.at[i,'VotosTotais'] = results.QT_VOTOS.sum()

In [ ]:
# df_results.sort_values(by='Bolsonaro_abslt')

### Check specific Sessao and Zona

In [ ]:
zona = 195
secao = 76
check_secao = df_ne[(df_ne['NR_ZONA']==zona)&(df_ne['NR_SECAO']==secao)][['NM_VOTAVEL','QT_VOTOS']].groupby(by='NM_VOTAVEL').sum("QT_VOTOS")
df_ne[(df_ne['NR_ZONA']==zona)&(df_ne['NR_SECAO']==secao)][['SG_UF','NM_MUNICIPIO','NM_VOTAVEL','QT_VOTOS']]

,SG_UF,NM_MUNICIPIO,NM_VOTAVEL,QT_VOTOS
4024778,BA,PILÃO ARCADO,LUIZ INÁCIO LULA DA SILVA,65
